In [115]:
# !pip install vaderSentiment
# !pip install imblearn
# !pip install xgboost

In [116]:
import numpy as np
import pandas as pd
# from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [31]:
# Health & Personal Care Category
# df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

# Watches Category
df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
df

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960199,US,39508379,REH8UQZAXQS40,B00005OT9I,2153632,Xhilaration&#174; Silver Dial Bangle Watch,Watches,2,17,17,N,N,Great prize - but size??,It's a great watch - but you don't have a chan...,2001-12-13
960200,US,44704251,RJ14QPZEOI9P8,B00005QEM6,903687397,Armitron Men's Titanium 3 Eye Sport Analog Chr...,Watches,5,9,10,N,N,A very classy watch!,"Well worth the price. This is a light weight, ...",2001-11-08
960201,US,40571775,R2B1G5650WWFCE,B00005QEME,252821780,Men's Timex Internet Messenger Sport Watch,Watches,5,3,16,N,N,This is a great watch,"Dear Targeteers,<BR>This watch is exelent. it ...",2001-11-06
960202,US,44474855,R2MMGPUWXXOFI2,B00004YK0H,118389241,Energizer 393 Button Cell Battery,Watches,4,0,0,N,N,Now watt a minute here.,"In the old days, the common hearing battery in...",2001-04-05


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
marketplace          960204 non-null object
customer_id          960204 non-null int64
review_id            960204 non-null object
product_id           960204 non-null object
product_parent       960204 non-null int64
product_title        960202 non-null object
product_category     960204 non-null object
star_rating          960204 non-null int64
helpful_votes        960204 non-null int64
total_votes          960204 non-null int64
vine                 960204 non-null object
verified_purchase    960204 non-null object
review_headline      960197 non-null object
review_body          960056 non-null object
review_date          960200 non-null object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [42]:
df_small = df[:20000]

# Deal with missing values for any of the columns we will be using
# df_small['star_rating'] = df_small['star_rating'].astype(int)
df_small['review_headline'] = df_small['review_headline'].fillna('.')
df_small['review_body'] = df_small['review_body'].fillna('.')
df_small['review_fulltext'] = df_small['review_headline'] + '. ' + df_small['review_body']

df_small['neg'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neg)
df_small['neu'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neu)
df_small['pos'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_pos)
df_small['compound'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_compound)

df_small.info()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 20 columns):
marketplace          20000 non-null object
customer_id          20000 non-null int64
review_id            20000 non-null object
product_id           20000 non-null object
product_parent       20000 non-null int64
product_title        20000 non-null object
product_category     20000 non-null object
star_rating          20000 non-null int64
helpful_votes        20000 non-null int64
total_votes          20000 non-null int64
vine                 20000 non-null object
verified_purchase    20000 non-null object
review_headline      20000 non-null object
review_body          20000 non-null object
review_date          20000 non-null object
review_fulltext      20000 non-null object
neg                  20000 non-null float64
neu                  20000 non-null float64
pos                  20000 non-null float64
compound             20000 non-null float64
dtypes: float64(4), int64(5), ob

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [43]:
analyzer = SentimentIntensityAnalyzer()

In [44]:
# Return dictionary of VADER sentiment scores for given sentence. Keys: 'neg', 'neu', 'pos', 'compound'
def sentiment_analyzer_scores(sentence, verbose=False):
    score = analyzer.polarity_scores(sentence)
    if verbose: print("{}".format(str(score)))
    
    return score
    
def sentiment_analyzer_scores_neg(sentence):
    return sentiment_analyzer_scores(sentence)['neg']

def sentiment_analyzer_scores_neu(sentence):
    return sentiment_analyzer_scores(sentence)['neu']

def sentiment_analyzer_scores_pos(sentence):
    return sentiment_analyzer_scores(sentence)['pos']

def sentiment_analyzer_scores_compound(sentence):
    return sentiment_analyzer_scores(sentence)['compound']

In [45]:
df_small['star_rating'].value_counts()
# df_small['verified_purchase'].value_counts()

5    12114
4     3316
1     1940
3     1579
2     1051
Name: star_rating, dtype: int64

In [46]:
df_numer = df_small.select_dtypes(include='number')
df_numer.info()
df_numer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
customer_id       20000 non-null int64
product_parent    20000 non-null int64
star_rating       20000 non-null int64
helpful_votes     20000 non-null int64
total_votes       20000 non-null int64
neg               20000 non-null float64
neu               20000 non-null float64
pos               20000 non-null float64
compound          20000 non-null float64
dtypes: float64(4), int64(5)
memory usage: 1.4 MB


,customer_id,product_parent,star_rating,helpful_votes,total_votes,neg,neu,pos,compound
0,3653882,937001370,5,0,0,0.000,0.616,0.384,0.8169
1,14661224,484010722,5,0,0,0.000,0.357,0.643,0.9531
2,27324930,361166390,2,1,1,0.000,1.000,0.000,0.0000
3,7211452,958035625,5,0,0,0.000,0.890,0.110,0.2732
4,12733322,765328221,4,0,0,0.024,0.715,0.261,0.9886
...,...,...,...,...,...,...,...,...,...
19995,32167840,354933056,5,0,0,0.000,0.754,0.246,0.7351
19996,2194392,649128875,4,0,0,0.068,0.777,0.155,0.4215
19997,15315799,811352940,5,0,0,0.000,0.682,0.318,0.4215
19998,40994978,714629498,5,0,0,0.000,0.517,0.483,0.4215


In [47]:
X = df_small.loc[:, ['helpful_votes', 'total_votes', 'neg', 'neu', 'pos', 'compound']]
y = df_small.loc[:, 'star_rating']

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [49]:
y_train.value_counts()

5    9072
4    2481
1    1470
3    1198
2     779
Name: star_rating, dtype: int64

In [50]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train) 
y_train_resampled.value_counts()

5    9072
4    9072
3    9072
2    9072
1    9072
Name: star_rating, dtype: int64

In [51]:
# Test out different classification algorithms
# Logistic Regression

logreg = LogisticRegression(fit_intercept=True, C=1e12, solver='liblinear', )
model_log = logreg.fit(X_train_resampled, y_train_resampled)
model_log

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
y_hat_test = logreg.predict(X_test)
y_hat_train = logreg.predict(X_train_resampled)

In [53]:
# We could subtract the two columns. If values or equal, difference will be zero. Then count number of zeros 
residuals = np.abs(y_train_resampled - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    18101
1    17739
2     7053
3     2126
4      341
Name: star_rating, dtype: int64
0    0.399052
1    0.391071
2    0.155489
3    0.046869
4    0.007518
Name: star_rating, dtype: float64


In [54]:
residuals = np.abs(y_test - y_hat_test)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    2738
1    1455
2     433
3     287
4      87
Name: star_rating, dtype: int64
0    0.5476
1    0.2910
2    0.0866
3    0.0574
4    0.0174
Name: star_rating, dtype: float64


In [57]:
# Confusion Matrix for training set - Logistic Regression
cf = confusion_matrix(y_train_resampled, y_hat_train)
cf

array([[5667, 1644,  970,  656,  135],
       [4098, 1936, 1536, 1078,  424],
       [2290, 1519, 1754, 1923, 1586],
       [ 495,  707, 1092, 2566, 4212],
       [ 206,  551,  422, 1715, 6178]])

In [65]:
print(classification_report(y_train_resampled, y_hat_train))

              precision    recall  f1-score   support

           1       0.44      0.62      0.52      9072
           2       0.30      0.21      0.25      9072
           3       0.30      0.19      0.24      9072
           4       0.32      0.28      0.30      9072
           5       0.49      0.68      0.57      9072

    accuracy                           0.40     45360
   macro avg       0.37      0.40      0.38     45360
weighted avg       0.37      0.40      0.38     45360



In [ ]:
# ROC Curve

In [75]:
# K Nearest Neighbors

# Instantiate KNeighborsClassifier
clf_knn = KNeighborsClassifier()

# Fit the classifier
clf_knn.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_knn = clf_knn.predict(X_train_resampled)

cm_knn = confusion_matrix(y_train_resampled, y_hat_knn)
cm_knn

array([[8082,  541,  245,  145,   59],
       [1520, 7048,  261,  154,   89],
       [1321,  472, 6428,  367,  484],
       [ 626,  377,  508, 6014, 1547],
       [ 768,  429,  659, 1350, 5866]])

In [72]:
print(classification_report(y_train_resampled, y_hat_knn))

              precision    recall  f1-score   support

           1       0.66      0.89      0.76      9072
           2       0.79      0.78      0.79      9072
           3       0.79      0.71      0.75      9072
           4       0.75      0.66      0.70      9072
           5       0.73      0.65      0.69      9072

    accuracy                           0.74     45360
   macro avg       0.74      0.74      0.74     45360
weighted avg       0.74      0.74      0.74     45360



In [ ]:
# Improve model performance by finding the best value for K

In [78]:
# Decision Trees

clf_dt = DecisionTreeClassifier(criterion='entropy')

clf_dt.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_dt = clf_dt.predict(X_train_resampled)

cm_dt = confusion_matrix(y_train_resampled, y_hat_dt)
cm_dt

array([[8091,  969,    1,    7,    4],
       [ 177, 8883,    1,    2,    9],
       [ 106,  907, 7884,  152,   23],
       [  10,  428,  101, 8163,  370],
       [  61,  447,  116,  424, 8024]])

In [79]:
print(classification_report(y_train_resampled, y_hat_dt))

              precision    recall  f1-score   support

           1       0.96      0.89      0.92      9072
           2       0.76      0.98      0.86      9072
           3       0.97      0.87      0.92      9072
           4       0.93      0.90      0.92      9072
           5       0.95      0.88      0.92      9072

    accuracy                           0.90     45360
   macro avg       0.92      0.90      0.91     45360
weighted avg       0.92      0.90      0.91     45360



In [91]:
# Bagged Trees

# Instantiate a BaggingClassifier
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_leaf_nodes=100), 
                                 n_estimators=20)

# Fit to the training data
bagged_tree.fit(X_train_resampled, y_train_resampled)

bagged_tree.score(X_train_resampled, y_train_resampled)

0.49852292768959433

In [92]:
y_hat_bagged = bagged_tree.predict(X_train_resampled)

cm_bagged = confusion_matrix(y_train_resampled, y_hat_bagged)
cm_bagged

array([[5543, 1987, 1081,  391,   70],
       [2174, 4787, 1214,  604,  293],
       [1381, 1843, 3339, 1685,  824],
       [ 419,  984, 1351, 3382, 2936],
       [ 191,  683,  661, 1975, 5562]])

In [93]:
print(classification_report(y_train_resampled, y_hat_bagged))

              precision    recall  f1-score   support

           1       0.57      0.61      0.59      9072
           2       0.47      0.53      0.49      9072
           3       0.44      0.37      0.40      9072
           4       0.42      0.37      0.40      9072
           5       0.57      0.61      0.59      9072

    accuracy                           0.50     45360
   macro avg       0.49      0.50      0.49     45360
weighted avg       0.49      0.50      0.49     45360



In [ ]:
# Random Forest

In [95]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth = 10)
forest.fit(X_train_resampled, y_train_resampled)
forest.score(X_train_resampled, y_train_resampled)

0.6006834215167548

In [96]:
y_hat_forest = forest.predict(X_train_resampled)

cm_forest = confusion_matrix(y_train_resampled, y_hat_forest)
cm_forest

array([[6109, 1816,  609,  445,   93],
       [1390, 6273,  617,  509,  283],
       [ 954, 1676, 4468, 1077,  897],
       [ 312, 1012,  935, 4262, 2551],
       [ 180,  694,  548, 1515, 6135]])

In [97]:
print(classification_report(y_train_resampled, y_hat_forest))

              precision    recall  f1-score   support

           1       0.68      0.67      0.68      9072
           2       0.55      0.69      0.61      9072
           3       0.62      0.49      0.55      9072
           4       0.55      0.47      0.50      9072
           5       0.62      0.68      0.64      9072

    accuracy                           0.60     45360
   macro avg       0.60      0.60      0.60     45360
weighted avg       0.60      0.60      0.60     45360



In [ ]:
# Grid Search / Random Search to find optimal parameters

In [ ]:
# SVM / Linear SVC

In [ ]:
# Naive Bayes?

In [ ]:
# SVC?

In [ ]:
# Ensemble Classifiers

In [99]:
# Adaboost
# Instantiate an AdaBoostClassifier
clf_ab = AdaBoostClassifier()

clf_ab.fit(X_train_resampled, y_train_resampled)
clf_ab.score(X_train_resampled, y_train_resampled)

0.4135361552028219

In [100]:
y_hat_ab = clf_ab.predict(X_train_resampled)

cm_ab = confusion_matrix(y_train_resampled, y_hat_ab)
cm_ab

array([[5671, 2031,  893,  381,   96],
       [4092, 2694, 1266,  729,  291],
       [2465, 1916, 1980, 1407, 1304],
       [ 788,  887, 1277, 2525, 3595],
       [ 627,  313,  604, 1640, 5888]])

In [101]:
print(classification_report(y_train_resampled, y_hat_ab))

              precision    recall  f1-score   support

           1       0.42      0.63      0.50      9072
           2       0.34      0.30      0.32      9072
           3       0.33      0.22      0.26      9072
           4       0.38      0.28      0.32      9072
           5       0.53      0.65      0.58      9072

    accuracy                           0.41     45360
   macro avg       0.40      0.41      0.40     45360
weighted avg       0.40      0.41      0.40     45360



In [102]:
# Instantiate an GradientBoostingClassifier
clf_gb = GradientBoostingClassifier()

clf_gb.fit(X_train_resampled, y_train_resampled)
clf_gb.score(X_train_resampled, y_train_resampled)

0.47872574955908287

In [103]:
y_hat_gb = clf_gb.predict(X_train_resampled)

cm_gb = confusion_matrix(y_train_resampled, y_hat_gb)
cm_gb

array([[5006, 2510,  981,  484,   91],
       [2230, 4487, 1328,  724,  303],
       [1283, 2270, 2888, 1642,  989],
       [ 312, 1100, 1260, 3366, 3034],
       [ 198,  646,  617, 1643, 5968]])

In [104]:
print(classification_report(y_train_resampled, y_hat_gb))

              precision    recall  f1-score   support

           1       0.55      0.55      0.55      9072
           2       0.41      0.49      0.45      9072
           3       0.41      0.32      0.36      9072
           4       0.43      0.37      0.40      9072
           5       0.57      0.66      0.61      9072

    accuracy                           0.48     45360
   macro avg       0.47      0.48      0.47     45360
weighted avg       0.47      0.48      0.47     45360



In [ ]:
# XGBoost

In [ ]:
# SGD?

In [ ]:
# Kernel Approximation?

In [24]:
df_watch = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [27]:
df_watch.loc[df_watch['star_rating'] == 1]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
7,US,49401598,R2RHFJV0UYBK3Y,B00A4EYBR0,844009113,INFANTRY Mens Night Vision Analog Quartz Wrist...,Watches,1,1,5,N,N,I was about to buy this thinking it was a ...,I was about to buy this thinking it was a Swis...,2015-08-31
11,US,16097204,R361XSS37V0NCZ,B0039UT5OU,685450910,Casio General Men's Watches Sporty Digital AE-...,Watches,1,0,0,N,N,I do not think this watch is a good product. D...,"I bought this watch on 2013, the screen had a ...",2015-08-31
37,US,10356300,R364RU42RZ606Z,B00F8AWMXA,397647155,Invicta Men's 15256 I-Force Gunmetal Ion-Plate...,Watches,1,0,0,N,Y,One Star,crap,2015-08-31
41,US,19983148,RQ70UFZVX7UVJ,B00S69FO0I,406011595,U.S. Polo Assn. Sport Men's US9506 Analog-Digi...,Watches,1,0,0,N,Y,poor quality,"Metal color was changed after the first use , ...",2015-08-31
53,US,1196727,R1SOUO6DCO4X5H,B00QBTQX50,29617990,iStrap Calfskin Leather Watch Band Replacement...,Watches,1,0,1,N,Y,One Star,It was bad it didnt fit my 22mm retro casio,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960179,US,52789057,R2SPXCB6TMIG7E,B00005QVXH,401515404,Timex Women's Fashion Watch 01017,Watches,1,2,2,N,N,Timepiece,Looks great! Does not keep time or have any in...,2002-10-22
960180,US,36726250,R1HEEW0HDPP0YH,B00005OT9I,2153632,Xhilaration&#174; Silver Dial Bangle Watch,Watches,1,4,4,N,N,Cheaply Made.....,This watch looks good &amp; is very attractive...,2002-10-12
960183,US,38105554,R1721LXOWD6H3C,B00005OT8Z,122254165,Timex Ironman Triathlon Digital Heart Rate Mon...,Watches,1,11,27,N,N,I was excited,I was very excited about this product when I p...,2002-09-18
960191,US,49987382,RPAY64ZST7TE3,B00005OT91,458246193,Timex Men's T44661 Expedition Metal Tech Watch,Watches,1,10,11,N,N,Piece of [stuff],It is suppossed to take a lickin.. BUT IT DOES...,2002-05-15


In [28]:
df_watch['star_rating'].value_counts()

5    571173
4    172781
1     85126
3     79116
2     52008
Name: star_rating, dtype: int64

In [15]:
# Clustering

from sklearn.cluster import KMeans

num_clusters = np.arange(2, 50)

# Set number of clusters at initialization time
for n in num_clusters:
    k_means = KMeans(n_clusters=n) 

    # Run the clustering algorithm
    k_means.fit(df_numer) 

    # Generate cluster index values for each row
    cluster_assignments = k_means.predict(df_numer)
    
    print(n, calinski_harabasz_score(df_numer, cluster_assignments))

NameError: name 'calinski_harabasz_score' is not defined

In [ ]:
cluster_assignments

In [ ]:
k_means.labels_

In [ ]:
# This code builds on the previous example
from sklearn.metrics import calinski_harabasz_score

# Note that we could also pass in k_means.labels_ instead of cluster_assignments
print(calinski_harabasz_score(df_numer, cluster_assignments))